In [9]:
!pip install selenium
!pip install beautifulsoup4

In [24]:
from selenium.webdriver.common.by import By
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import datetime
import requests
import pandas as pd
from selenium.webdriver.chrome.service import Service

# url
csv_file_path = 'C:/Users/lgsc0/OneDrive/바탕 화면/신기플/cafe_id_test.csv'  # CSV 파일 경로
df = pd.read_csv(csv_file_path)  # CSV 파일 읽기
url_cafe_ids = df['id_value'].tolist()  # CSV에서 'id_value' 열을 리스트로 변환
base_url = 'https://m.place.naver.com/restaurant/{}'  # 기본 URL 형식

# WebDriver 설정
options = webdriver.ChromeOptions()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')  # 브라우저를 백그라운드에서 실행
chrome_options.add_argument('--no-sandbox')  # 샌드박스 모드 비활성화
chrome_options.add_argument('--disable-dev-shm-usage')  # 공유 메모리 사용 비활성화
chrome_options.add_argument('lang=ko_KR')  # 언어 설정 (한국어)
chrome_options.add_argument('window-size=1920x1080')  # 윈도우 크기 설정
chrome_options.add_argument("disable-gpu")  # GPU 비활성화

# WebDriver 초기화
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# BS4 설정 (Secondary Access를 위한 설정)
session = requests.Session()
headers = {
    "User-Agent": "user value"  # User-Agent 설정
}

retries = Retry(total=5,  # 최대 재시도 횟수
                backoff_factor=0.1,  # 재시도 간 대기 시간 증가 비율
                status_forcelist=[500, 502, 503, 504])  # 재시도 대상 상태 코드 리스트

session.mount('http://', HTTPAdapter(max_retries=retries))  # HTTPAdapter에 재시도 설정 적용

# 크롤링 결과를 저장할 리스트 초기화
results = []

# 크롤링/스크래핑 시작
try:
    # URL 생성 및 리스트에 추가
    target_url = []
    for cafe_id in url_cafe_ids:
        url = base_url.format(cafe_id)  # cafe_id를 사용하여 URL 생성
        print(f"Processing URL: {url}")
        target_url.append(url)  # 생성된 URL을 리스트에 추가

    for url in target_url:
        print(f"Accessing URL: {url}")
        driver.get(url)  # WebDriver로 URL 접속

        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
        res = driver.get(url)
        driver.implicitly_wait(5)  # 최대 10초 동안 대기

        # 페이지 다운 (스크롤)
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)

        html = driver.page_source  # 페이지 소스 가져오기
        bs = BeautifulSoup(html, 'html.parser')  # BeautifulSoup 객체 생성

        try:
            for i in range(3):  # 클릭 횟수 설정 (여기서는 3번)
                driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').click()
                time.sleep(0.5)  # 클릭 간 대기 시간
        except Exception as e:
            print('finish')  # 클릭 실패 시 출력
        else:
            print('times_finish')  # 클릭 성공 시 출력

        time.sleep(5)  # 추가 대기 시간
        html = driver.page_source  # 페이지 소스 다시 가져오기
        bs = BeautifulSoup(html, 'html.parser')  # BeautifulSoup 객체 재생성

        # 카페 이름 가져오기
        try:
            cafe_element = driver.find_element(By.XPATH, '//*[@id="_title"]/div/span[1]')
            cafe = cafe_element.text if cafe_element else ''  # 요소가 존재하면 텍스트, 없으면 빈 문자열
        except Exception as e:
            cafe = ''

        # 주소 가져오기
        try:
            address_element = driver.find_element(By.CSS_SELECTOR, 'div.O8qbU.tQY7D > div.vV_z_ > a.PkgBl > span.LDgIH')
            address = address_element.text if address_element else ''  # 요소가 존재하면 텍스트, 없으면 빈 문자열
        except Exception as e:
            address = ''
    
        # 평점 가져오기
        try:
            # 1. total_review와 rating이 있는 메뉴바 xpath 탐색
            menubar_element = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]')
    
            # 2. 하위 span 요소들의 개수를 확인하여 분기 처리
            spans = menubar_element.find_elements(By.XPATH, './span')

            if len(spans) == 2:
                # i의 범위가 1~2라면 rating이 없으므로 total_review만 추출
                total_review_element = spans[0]  # 첫 번째 span에서 total_review 추출
                total_review = total_review_element.text if total_review_element else ''
                total_review = total_review.replace('방문자 리뷰 ', '').replace(',', '')  # 문자열 제거
                total_review = float(total_review) if total_review else None
                rating = None  # rating은 없음

            elif len(spans) == 3:
                # i의 범위가 1~3이라면 rating과 total_review 둘 다 추출
                rating_element = spans[0]  # 첫 번째 span에서 rating 추출
                rating = rating_element.text if rating_element else ''
                rating = rating.replace('별점', '')  # "별점" 문자열 제거
                rating = float(rating) if rating else None
        
                total_review_element = spans[1]  # 두 번째 span에서 total_review 추출
                total_review = total_review_element.text if total_review_element else ''
                total_review = total_review.replace('방문자 리뷰 ', '').replace(',', '')  # 문자열 제거
                total_review = float(total_review) if total_review else None

            else:
                # 예상치 못한 경우 처리
                rating = None
                total_review = None

        except Exception as e:
            rating = None
            total_review = None


        # 설명 가져오기
        try:
            short_description_element = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[2]/div[1]/div[2]/div')
            short_description = short_description_element.text if short_description_element else ''  # 요소가 존재하면 텍스트, 없으면 빈 문자열
        except Exception as e:
            short_description = ''
    
        # 링크 가져오기
        try:
            sns_element = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div/div[2]/div[1]/div/div[4]/div/div[2]/span/a')
            sns_link = sns_element.get_attribute('href') if sns_element else ''  # 요소가 존재하면 href 속성, 없으면 빈 문자열

            if 'instagram' not in sns_link:
                sns_link = ''
                
        except Exception as e:
            sns_link = ''

        # 정보 버튼 클릭 및 정보 가져오기
        try:
            # 1. 메뉴바 xpath 탐색
            menubar_element = driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[4]/div/div/div/div')

            # 2. 하위 클래스 중 '정보' 텍스트가 있는 클래스 탐색
            info_button = None
            menu_items = menubar_element.find_elements(By.TAG_NAME, 'a')  # 메뉴바의 모든 'a' 태그 요소 탐색
            for item in menu_items:
                if '정보' in item.text:  # 텍스트가 '정보'인 버튼 찾기
                    info_button = item
                    break

            # 3. '정보' 텍스트를 담은 버튼 클릭
            if info_button:
                info_button.click()  # 정보 버튼 클릭
                time.sleep(3)  # 로딩할 때까지 잠시 대기
            else:
                print("정보 버튼을 찾을 수 없습니다.")  # '정보' 버튼이 없을 경우 메시지 출력

            # info 요소를 selector path로 찾기
            try:
                info_element = driver.find_element(By.CSS_SELECTOR, '#app-root > div > div > div > div:nth-child(6) > div > div.place_section.no_margin.Od79H > div > div > div.Ve1Rp > div')
                info = info_element.text if info_element else ''  # info 요소가 있으면 텍스트 추출, 없으면 빈칸
            except Exception as e:
                info = ''

        except Exception as e:
            print(f"Error retrieving info: {e}")
            info = ''  # 예외 발생 시 info를 빈 문자열로 설정

        # 크롤링 결과 리스트에 추가
        results.append([cafe, address, short_description, rating, total_review, sns_link, info])

except Exception as e:
    print(e)  # 예외 발생 시 출력

# 크롤링 결과를 DataFrame으로 변환
df_results = pd.DataFrame(results, columns=['cafe', 'address', 'short_description', 'rating', 'total_review', 'sns_link', 'info'])

# 현재 시간 기준으로 파일명 생성
file_name = 'cafe_info_crawling.csv'

# DataFrame을 CSV 파일로 저장
df_results.to_csv(file_name, index=False, encoding='utf-8-sig')  # 'utf-8-sig'는 Excel에서 한글이 깨지지 않도록 설정


Processing URL: https://m.place.naver.com/restaurant/742676592
Processing URL: https://m.place.naver.com/restaurant/1092567422
Accessing URL: https://m.place.naver.com/restaurant/742676592
finish
Accessing URL: https://m.place.naver.com/restaurant/1092567422


KeyboardInterrupt: 